In [344]:
import pandas as pd
import numpy as np
import vnstock as vn
import calculation as cal

In [345]:
data = cal.DataProcessor.load_data('VCB') 
# data = data.set_index('time')
#Tính trung bình 20 phiên gần nhất
mean_20 = data['volume'].rolling(window=20).mean() 
#Tính on-balance volume
data['on-balance_volume'] = data['volume']/mean_20 

data['OBV_label'] = data['on-balance_volume'].apply(cal.DataProcessor.label_values)
#Giá đóng cửa ngày t-1
data['close_t_minus_1'] = data['close'].shift(1) 
data['bar_type'] = data.apply(cal.DataProcessor.compare_close_prices, axis=1)   
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])

data['label_spread'] = data['para'].apply(cal.DataProcessor.label_spread)
data['close_bar_label'] = data.apply(cal.DataProcessor.label_close_bar, axis=1)


In [346]:

# df_filtered = data[data['bar_type'] == 'up-bar']
# df_filtered


# Pattern of Weakness

## Weakness A
- down bar
- high volume
- medium or low spread
- close: middle-third or bottom-third

In [347]:
def is_weakness_a_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['OBV_label']=='high') and \
        ((data['label_spread'] == 'low') or (data['label_spread']=='medium')) and \
        ((data['close_bar_label'] == 'middle-third') or (data['close_bar_label']=='bottom-third')):
        return True
    else:
        return False

In [348]:
# Sample data
sample_data = {
    'bar_type': 'down-bar',
    'OBV_label': 'high',
    'label_spread': 'low',
    'close_bar_label': 'middle-third'
}
result = is_weakness_a_signal(sample_data)
print("Is weakness a signal?", result)

Is weakness a signal? True


## No demand - Weakness B
- up bar
- volume: low or high
- spread: low
- close: bottom-third and middle-third

In [349]:
def is_no_demand_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        (data['label_spread'] == 'low') and \
        ((data['close_bar_label'] == 'bottom-third') or data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False

In [350]:
result = is_no_demand_signal(sample_data)
print("Is no demand signal?", result)

Is no demand signal? False


## Up-trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high or low volume


In [351]:
def is_up_trust_signal(data):
    if ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        (data['label_spread'] == 'high') and \
        ((data['close_bar_label'] == 'bottom-third') ):
        return True
    else:
        return False

## Stop volume
- up bar
- high spread
- high volume

In [352]:
def is_stop_volume_signal(data):
    if (data['bar_type']=='up-bar') and \
        (data['OBV_label']=='high') and \
        (data['label_spread'] == 'high'):
        return True
    else:
        return False

In [353]:

for i in range(len(data)):

    if i % 4 == 0:
        data.loc[i, 'results'] = is_weakness_a_signal(data.loc[i])
    elif i % 4 == 1:
        data.loc[i, 'results'] = is_no_demand_signal(data.loc[i])
    elif i % 4 == 2:
        data.loc[i, 'results'] = is_up_trust_signal(data.loc[i])
    elif i % 4 == 3:
        data.loc[i, 'results'] = is_stop_volume_signal(data.loc[i])
    
data

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,False
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium,top-third,False
1270,2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium,bottom-third,False
1271,2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low,top-third,False
1272,2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458,low,91000.0,down-bar,1.000000,high,bottom-third,False


In [354]:
data['results'].value_counts()

results
False    1211
True       63
Name: count, dtype: int64

In [355]:
# Tạo cột "Lệnh"
data['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

window_size =4  # Kích thước cửa sổ

data['rolling_sum'] = data['results'].rolling(window_size).sum()

data.loc[data['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

data = data.drop(columns='rolling_sum')

data.head(5)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False,hold
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False,hold
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False,hold
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,False,hold
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False,hold


In [356]:
# Example
# # Tạo DataFrame mẫu
# data1 = {'ngay': ['2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28', '2024-03-01', '2024-03-02','2024-03-03', '2024-03-04', '2024-03-05'],
#         'results': [False, True, True, True, True, True, False, True, True]}
# df = pd.DataFrame(data1)
# # Tạo cột "Lệnh"
# df['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

# window_size = 4  # Kích thước cửa sổ

# df['rolling_sum'] = df['results'].rolling(window_size).sum()

# df.loc[df['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

# df = df.drop(columns='rolling_sum')

# print(df)

In [357]:
data[['time', 'OBV_label', 'bar_type', 'label_spread', 'close_bar_label', 'results', 'execute_a_trade']].head(5)

,time,OBV_label,bar_type,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,NaN,down-bar,medium,bottom-third,False,hold
1,2019-01-03,NaN,down-bar,low,middle-third,False,hold
2,2019-01-04,NaN,up-bar,medium,top-third,False,hold
3,2019-01-07,NaN,up-bar,high,bottom-third,False,hold
4,2019-01-08,NaN,up-bar,high,top-third,False,hold


In [358]:
data['execute_a_trade'].value_counts()

execute_a_trade
hold    1274
Name: count, dtype: int64

# Power pattern

## Power A
- up bar
- medium spread
- medium or high volume
- close: top-third

In [359]:
def is_power_A_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        (data['label_spread'] == 'medium') and \
        ((data['OBV_label']=='medium') or (data['OBV_label'] == 'low')) and \
        (data['close_bar_label'] == 'top-third'):
        return True
    else:
        return False

## Power B - Lack of order
- down bar 
- low spread 
- low or high volume
- close: top-third or bottom-third

In [360]:
def is_power_B_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        ((data['close_bar_label'] == 'top-third') or (data['close_bar_label'] == 'bottom-third')):
        return True
    else:
        return False

## Reverse Up-Trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high and low volume


In [361]:
def is_reverse_up_trust_signal(data):
    if ((data['bar_type'] == 'down-bar') or (data['bar_type']) == 'up-bar') and \
        (data['label_spread'] == 'high') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        (data['close_bar_label'] == 'bottom-third'):
        return True
    else:
        return False

## Stopped volume
- down bar
- low spread
- close: middle-third
- high volume


In [362]:
def is_stopped_volume_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        (data['OBV_label'] == 'high') and \
        (data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False